In [75]:
import pandas as pd


data = pd.read_csv('../networks/data/global_before_1700_before_writers.csv', index_col=[0])
data['meta_occupation'] = data['meta_occupation'].apply(lambda x: x.split(' | '))
data = data.explode('meta_occupation')
data = data.reset_index(drop=True)
before_adding_writers_only = data.meta_occupation.value_counts()
before_adding_writers_only = before_adding_writers_only.reset_index()
before_adding_writers_only.columns = ['meta_occupation', 'inital_count']

In [76]:
len(set(data.meta_occupation))
list(set(data.meta_occupation))

['geographer',
 'philosopher',
 'political scientist',
 'economist',
 'theologian',
 'physicist',
 'archeologist',
 'anatomist',
 'sociologist',
 'art theorist',
 'mathematician',
 'botanist',
 'chemist',
 'meteorologist',
 'logician',
 'epidemiologist',
 'pharmacologist',
 'anthropologist',
 'geologist',
 'linguist',
 'zoologist',
 'paleontologist',
 'astronomer',
 'musicologist',
 'historian']

In [77]:
import sys

sys.path.append("../")

import sqlite3
import pandas as pd
from functions.env import DB_SCIENCE_PATH_NEW

data_path = "/Users/charlesdedampierre/Desktop/cultura_project/cultura.db"

conn = sqlite3.connect(data_path)
new_conn = sqlite3.connect(DB_SCIENCE_PATH_NEW)
df_occupations = pd.read_sql("SELECT * FROM individual_occupations", conn)
df_occupations = df_occupations[['occupations_name', 'occupations_category']].drop_duplicates()
#df_occupations = df_occupations[~df_occupations['occupations_category'].str.contains('artist')]
set(df_occupations.occupations_category)

df_occupations = df_occupations.rename(columns={'occupations_name':'meta_occupation'})
df_only_writers = df_occupations[df_occupations['meta_occupation'].isin(list(set(data.meta_occupation)))]
df_only_writers = df_only_writers[df_only_writers['occupations_category']=='writer']

In [78]:


only_writers_occupations = list(df_only_writers.meta_occupation)
only_writers_occupations

['philosopher', 'theologian', 'historian', 'musicologist', 'linguist']

### if we add them

In [79]:
df_ind = pd.read_sql("SELECT * FROM individual_occupations", conn)
df_ind = df_ind[~df_ind["occupations_category"].str.contains("artist")]

df_ind_birth = pd.read_sql("SELECT * FROM individuals_main_information", conn)
df_ind_birth = df_ind_birth[df_ind_birth['birthyear']<=1700]
df_ind_birth = df_ind_birth[['individual_wikidata_id', 'birthyear']]

df_cultura = pd.merge(df_ind_birth, df_ind, on = 'individual_wikidata_id')
df_cultura = df_cultura.rename(columns={'occupations_name':'meta_occupation'})
df_cultura = df_cultura[df_cultura['meta_occupation'].isin(only_writers_occupations)]
total_occupations_only_writers = df_cultura.meta_occupation.value_counts()
total_occupations_only_writers = total_occupations_only_writers.reset_index()
total_occupations_only_writers.columns = ['meta_occupation', 'count_total']

In [80]:
df_cultura[df_cultura['meta_occupation']=='theologian']

,individual_wikidata_id,birthyear,individual_name,occupations_wikidata_id,meta_occupation,occupations_category
0,Q2822993,1100.0,Achard of Saint-Victor,Q1234713,theologian,writer
2,Q1349852,1689.0,Jakob Wilhelm Feuerlein,Q1234713,theologian,writer
14,Q38541085,1589.0,Bésian Arroy,Q1234713,theologian,writer
22,Q742311,634.0,Chad of Mercia,Q1234713,theologian,writer
24,Q67820,1532.0,Bartholomäus Ringwaldt,Q1234713,theologian,writer
...,...,...,...,...,...,...
46579,Q561667,1259.0,Ubertino of Casale,Q1234713,theologian,writer
46607,Q3939246,1290.0,Roberto de' Bardi,Q1234713,theologian,writer
46609,Q434550,1380.0,Jerome of Prague,Q1234713,theologian,writer
46612,Q733598,1522.0,Moses ben Jacob Cordovero,Q1234713,theologian,writer


In [84]:
final_analysis_writers = pd.merge(before_adding_writers_only, total_occupations_only_writers, on = 'meta_occupation')
final_analysis_writers.sort_values('count_total', ascending=False)

,meta_occupation,inital_count,count_total
3,theologian,270,6305
1,historian,474,4400
0,philosopher,528,3155
4,linguist,172,719
2,musicologist,288,251
